In [1]:
import pandas as pd
import numpy as np
import datetime as dt

'''
Instructions: youll need to put the exchange rate table "CurrencyTable" into the same folder as your
existing kickstarter file. The code that transforms the data takes forever finish - at least 3 mins.
'''

#change the path that the read_csv is looking at
df = pd.read_csv('CurrencyTable.csv',index_col=0)
df.index = pd.to_datetime(df.index)
new_index = pd.date_range(df.index.min(),df.index.max(),freq='MS')
df=df.reindex(new_index)
df=df.interpolate().unstack()
rate = pd.DataFrame(df)
rate.columns = ['rate']
rate.rename_axis(['currency','rtdate']).head()
#rate = df.interpolate().unstack().rename_axis(['currency','rtdate']).reset_index()

rate
currency rtdate              
AUD      2010-01-01  0.897274
         2010-02-01  0.896608
         2010-03-01  0.895943
         2010-04-01  0.895277
         2010-05-01  0.894612

In [2]:
#Import the data set into dataframe
#Change the path that read_csv is looking at
tdf = pd.read_csv('ksdata.csv', index_col=0)

In [3]:
'''
Converting the dataset
'''

#Convert columns to datetime
tdf.launched = pd.to_datetime(tdf.launched)
tdf.deadline = pd.to_datetime(tdf.deadline)

#Drop the columns where the launch date is 1970 - 5 itmes total
tdf = tdf.drop(tdf[tdf.launched < '2000-01-01'].index)

#create a column that gives you the duration of the campaign in days
tdf['date_diff'] = tdf.deadline - tdf.launched
tdf.date_diff = tdf.date_diff.apply(lambda x: x.days)

#change the country code for N,0" error to first two letters of currency
tdf['country'] = np.where(tdf['country'] == 'N,0"',tdf.currency.str[:2],tdf['country'])

#create new column
tdf['successful'] = tdf.state

#change funding_status to either 'successful' or 'failed' based on pledged > goal
tdf['successful'] = np.where(tdf['usd_pledged_real'] >= tdf['usd_goal_real'], 1 , 0)

#added a new column to dataframe that matches other dataframe for join
tdf['rtdate'] = tdf['launched'].values.astype('datetime64[M]')

#drop usd_goal_real and usd_pledged_real
tdf.drop(['usd_goal_real','usd_pledged_real','usd pledged'], axis=1, inplace=True)


In [4]:
result = tdf.join(rate, on=['currency', 'rtdate'], how='left')

In [5]:
result['goal_usd'] = result.goal * result.rate
result['pledged_usd'] = result.pledged * result.rate

In [6]:
result.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,date_diff,successful,rtdate,rate,goal_usd,pledged_usd
ID,,,,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,58,0,2015-08-01,1.527486,1527.485876,0.0
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,59,0,2017-09-01,1.000000,30000.000000,2421.0
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,44,0,2013-01-01,1.000000,45000.000000,220.0
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,29,0,2012-03-01,1.000000,5000.000000,1.0
1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,55,0,2015-07-01,1.000000,19500.000000,1283.0


# Feature engineering 
## Text mining of project names

In [7]:
pd.set_option('display.max_colwidth', -1)

In [8]:
index = range(len(result))
values = result.values
columns = result.columns
df = pd.DataFrame(values, index=index, columns = columns)

In [9]:
df.to_csv('data_textmining.csv')

In [10]:
dfname = df

In [11]:
dfname.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,date_diff,successful,rtdate,rate,goal_usd,pledged_usd
0,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 00:00:00,1000,2015-08-11 12:12:28,0,failed,0,GB,58,0,2015-08-01 00:00:00,1.52749,1527.49,0
1,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01 00:00:00,30000,2017-09-02 04:43:57,2421,failed,15,US,59,0,2017-09-01 00:00:00,1,30000,2421
2,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:00:00,45000,2013-01-12 00:20:50,220,failed,3,US,44,0,2013-01-01 00:00:00,1,45000,220
3,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 00:00:00,5000,2012-03-17 03:24:11,1,failed,1,US,29,0,2012-03-01 00:00:00,1,5000,1
4,Community Film Project: The Art of Neighborhood Filmmaking,Film & Video,Film & Video,USD,2015-08-29 00:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,55,0,2015-07-01 00:00:00,1,19500,1283


In [12]:
import re

In [13]:
# make sure all names are string
dfname['name'] = dfname.name.astype(str)

In [14]:
# there are 23105  projects with (canceled) in the name
pattern = '\(canceled\)'
dfname['name'] = dfname.name.str.replace(pattern, '', n=1, case=False)

In [15]:
# there are 1060  projects with (suspended) in the name
pattern = '\(suspended\)'
dfname['name'] = dfname.name.str.replace(pattern, '', n=1, case=False)

In [16]:
#remove rare characters 
pattern = '[^A-Za-z0-9!? ]'
dfname['name'] = dfname.name.str.replace(pattern, '')

In [17]:
pattern = '\(\)'
dfname['name'] = dfname.name.str.replace(pattern, '')

In [18]:
dfname.shape

(378654, 17)

In [19]:
#remove excess whitespace
for i in range(8):
    pattern = '  '
    dfname['name'] = dfname.name.str.replace(pattern, ' ')

In [21]:
pattern = '  '
dfname.name.str.find(pattern).sum()

-378654

In [22]:
# create new feature: name length
dfname['name_len'] = dfname.name.str.len()

In [26]:
# convert all to lowercase before split
dfname['name_low'] = dfname.name.str.lower()

In [27]:
# get rid of ? ! before split
pattern = '[!?]'
dfname['name_low'] = dfname.name_low.str.replace(pattern, '')

In [28]:
# drop empty names
dfname = dfname.drop(dfname[dfname.name_len == 0].index, axis=0)
dfname = dfname.drop(dfname[dfname.name == ' '].index, axis=0) 

In [29]:
# split each name by whitespace creating a list with words
dfname['name_words']  = dfname.name_low.str.strip().str.split('[\s]')

In [30]:
# create new feature: number of words in each name
dfname['number_words']  =  dfname.name_words.str.len()

In [31]:
dfname.describe()

,name_len,number_words
count,378650.000000,378650.000000
mean,32.902049,5.375339
std,15.203882,2.590772
min,1.000000,1.000000
25%,20.000000,3.000000
50%,32.000000,5.000000
75%,46.000000,7.000000
max,85.000000,27.000000


In [32]:
dfname.name_words.isnull().sum()

0

In [33]:
# create a safe copy
table2 = dfname

In [34]:
rows= list()
for row in table2[['successful', 'name_words']].iterrows():
    r = row[1]
    for word in r.name_words:
        rows.append((r.successful, word))
        
words = pd.DataFrame(rows, columns=['successful', 'word'])
words.head()

,successful,word
0,0,the
1,0,songs
2,0,of
3,0,adelaide
4,0,abullah


In [35]:
# drop empty words
words = words.drop(words[words.word == ''].index, axis=0)

In [36]:
# pivot table
count_words = words.pivot_table(index = 'word', columns='successful',aggfunc=np.size)

In [39]:
count_words = count_words.sort_values(1, ascending=False)

In [40]:
count_words

successful,0,1
word,,
the,127362.0,82720.0
a,59404.0,42984.0
of,46576.0,28572.0
and,32742.0,18362.0
for,31512.0,16790.0
album,11628.0,16426.0
to,26896.0,14756.0
new,13696.0,14036.0
in,21820.0,13920.0


In [41]:
# save to csv for further analysis
count_words.to_csv('count_words.csv')

In [42]:
# create new features
n_successful = len(dfname[dfname.successful == 1])

n_failed = len(dfname[dfname.successful == 0])

count_words['prop_successful'] = count_words[1] / n_successful

count_words['prop_failed'] = count_words[0] / n_failed

In [43]:
count_words['diff'] = count_words['prop_successful'] - count_words['prop_failed']

In [44]:
sorted_bydiff = count_words.sort_values(by='diff', ascending=False)

In [45]:
sorted_bydiff

successful,0,1,prop_successful,prop_failed,diff
word,,,,,
the,127362.0,82720.0,0.603611,0.527143,0.076467
album,11628.0,16426.0,0.119861,0.048128,0.071734
a,59404.0,42984.0,0.313656,0.245869,0.067786
new,13696.0,14036.0,0.102421,0.056687,0.045734
film,11618.0,11644.0,0.084967,0.048086,0.036880
debut,3840.0,6138.0,0.044789,0.015894,0.028896
by,10274.0,9764.0,0.071248,0.042523,0.028725
short,4620.0,6236.0,0.045504,0.019122,0.026382
ep,4056.0,5692.0,0.041535,0.016788,0.024747


In [92]:
dfname.shape

(378651, 21)

In [116]:
dfname[['successful', 'number_words']].groupby('successful').number_words.mean()

successful
0    5.170068
1    5.735001
Name: number_words, dtype: float64

In [117]:
dfname[['successful', 'name_len']].groupby('successful').mean()

,name_len
successful,
0,31.723806
1,34.636403
